<a href="https://colab.research.google.com/github/Andrea-1704/AML-labs/blob/main/rel_trial_maskesd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries to install

In [ ]:
# !pip install torch==2.6.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# !pip install pyg-lib -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-geometric==2.6.0 -f https://data.pyg.org/whl/torch-2.6.0+cu118.html

# !pip install pytorch_frame[full]==1.2.2
# !pip install relbench[full]==1.0.0

# Import

In [ ]:
import os
import torch
import relbench
import numpy as np
from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task
import math
from tqdm import tqdm
import torch_geometric
import torch_frame
from torch_geometric.seed import seed_everything
from relbench.modeling.utils import get_stype_proposal
from collections import defaultdict
import requests
from io import StringIO
from torch_frame.config.text_embedder import TextEmbedderConfig
from relbench.modeling.graph import make_pkey_fkey_graph
from torch.nn import BCEWithLogitsLoss
import copy
from typing import Any, Dict, List
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_geometric.typing import NodeType
from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder
from relbench.modeling.graph import get_node_train_table_input, make_pkey_fkey_graph
from torch_geometric.loader import NeighborLoader
import pyg_lib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import ModuleDict, Linear
import torch.nn.functional as F
from torch import nn

from torch_frame.data.stats import StatType
import random
import torch.nn as nn
import torch.nn.functional as F
import torch
from typing import Dict, List, Tuple
from torch import Tensor

# Dataset and Task definition

In [ ]:
def get_stype_enum_with_value(full_stype_dict, target_value: str):
    for table_entry in full_stype_dict.values():
        if isinstance(table_entry, dict):
            for val in table_entry.values():
                if str(val) == target_value:
                    return val
    raise ValueError(f"Tipo '{target_value}' non trovato nei tipi esistenti")


In [ ]:
from torch_frame.data.stats import StatType

def merge_text_columns_to_categorical(db, stype_dict):
    for table_name in db.table_dict:
        table = db.table_dict[table_name]
        new_colname = "merged_text_id"

        # Cerca colonne di tipo test_embedded
        col_to_type = stype_dict[table_name]
        all_text_cols = [
            col for col, stype in col_to_type.items()
            if str(stype) == "text_embedded"
        ]

        if not all_text_cols:
            continue

        # Ordina le colonne alfabeticamente per avere ordine stabile
        sorted_cols = sorted(set(all_text_cols))

        # Combina i valori riga per riga
        merged_col = table.df[sorted_cols].astype(str).apply(lambda row: "_".join(row), axis=1)

        # Elimina i vecchi tipi dallo stype_dict
        for col in sorted_cols:
            if col in stype_dict[table_name]:
                del stype_dict[table_name][col]

        # Elimina le colonne anche dal DataFrame
        table.df.drop(columns=sorted_cols, inplace=True, errors="ignore")

        # Aggiungi la colonna fusa
        table.df[new_colname] = merged_col

        # Assegna il tipo corretto
        #stype_dict[table_name][new_colname] = stype.categorical
        categorical_type = get_stype_enum_with_value(stype_dict, "categorical")
        stype_dict[table_name][new_colname] = categorical_type


    return db, stype_dict


In [ ]:
dataset = get_dataset("rel-trial", download=True)
task = get_task("rel-trial", "study-adverse", download=True)

train_table = task.get_table("train")
val_table = task.get_table("val")
test_table = task.get_table("test")

out_channels = 1
loss_fn = L1Loss()
# this is the mae loss and is used when have regressions tasks.
tune_metric = "mae"
higher_is_better = False

#seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
root_dir = "./data"

db = dataset.get_db() #get all tables
col_to_stype_dict = get_stype_proposal(db)
#this is used to get the stype of the columns

cuda
Loading Database object from /root/.cache/relbench/rel-trial/db...
Done in 8.25 seconds.


In [ ]:
db_nuovo, col_to_stype_dict_nuovo = merge_text_columns_to_categorical(db, col_to_stype_dict)

# Embedder

In [ ]:
# text_embedder_cfg = TextEmbedderConfig(
#     text_embedder=LightweightGloveEmbedder(device=device), batch_size=256
# )

data, col_stats_dict = make_pkey_fkey_graph(
    db_nuovo,
    col_to_stype_dict=col_to_stype_dict_nuovo,
    #text_embedder_cfg=text_embedder_cfg,
    text_embedder_cfg = None,
    cache_dir=None  # ← forzatamente disabilitata
)

# Graph Loader

In [ ]:
loader_dict = {}

for split, table in [
    ("train", train_table),
    ("val", val_table),
    ("test", test_table),
]:
    table_input = get_node_train_table_input(
        table=table,
        task=task,
    )

    entity_table = table_input.nodes[0]

    loader_dict[split] = NeighborLoader(
        data,
        num_neighbors=[
            128 for i in range(2) #################################
        ],
        time_attr="time",
        input_nodes=table_input.nodes,
        input_time=table_input.time,
        transform=table_input.transform,
        batch_size=512,
        temporal_strategy="uniform",
        shuffle=split == "train",
        num_workers=0,
        persistent_workers=False,
    )

In [ ]:
# qui i parametri di train_table, val_table, test_table, task e data sono
#parametri globali

def loader_dict_fn(batch_size, num_neighbours):
    loader_dict = {}

    for split, table in [
        ("train", train_table),
        ("val", val_table),
        ("test", test_table),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )

        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[num_neighbours for _ in range(2)],
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            batch_size=batch_size,
            temporal_strategy="uniform",
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )

    return loader_dict


# Model

In [ ]:
class Model(torch.nn.Module):

    def __init__(
        self,
        data: HeteroData,
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        shallow_list: List[NodeType] = [],
        id_awareness: bool = False,
    ):
        super().__init__()

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict={
                node_type: data[node_type].tf.col_names_dict
                for node_type in data.node_types
            },
            node_to_col_stats=col_stats_dict,
        )

        self.temporal_encoder = HeteroTemporalEncoder(
            node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
        )

        self.gnn = HeteroGraphSAGE(
            node_types=data.node_types,
            edge_types=data.edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1, ###################################################
        )
        self.embedding_dict = ModuleDict(
            {
                node: Embedding(data.num_nodes_dict[node], channels)
                for node in shallow_list
            }
        )

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = torch.nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for embedding in self.embedding_dict.values():
            torch.nn.init.normal_(embedding.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()

    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType,
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return self.head(x_dict[entity_table][: seed_time.size(0)])

    def encode_node_types(self, batch: HeteroData, node_types: List[str]) -> Dict[str, Tensor]:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(seed_time, batch.time_dict, batch.batch_dict)

        for node_type in node_types:
            if node_type in rel_time_dict:
                x_dict[node_type] = x_dict[node_type] + rel_time_dict[node_type]
            if node_type in self.embedding_dict:
                x_dict[node_type] = x_dict[node_type] + self.embedding_dict[node_type](batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return {ntype: x_dict[ntype] for ntype in node_types if ntype in x_dict}



    def forward_dst_readout(
        self,
        batch: HeteroData,
        entity_table: NodeType,
        dst_table: NodeType,
    ) -> Tensor:
        if self.id_awareness_emb is None:
            raise RuntimeError(
                "id_awareness must be set True to use forward_dst_readout"
            )
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)
        x_dict[entity_table][: seed_time.size(0)] += self.id_awareness_emb.weight

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
        )

        return self.head(x_dict[dst_table])

# Training functions

In [ ]:
def train(model, optimizer, loader_dict) -> float:
    model.train()

    loss_accum = count_accum = 0
    for batch in tqdm(loader_dict["train"]):
        batch = batch.to(device)

        optimizer.zero_grad()
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred

        loss = loss_fn(pred.float(), batch[entity_table].y.float())
        loss.backward()
        optimizer.step()

        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    return loss_accum / count_accum


@torch.no_grad()
def test(model, loader: NeighborLoader) -> np.ndarray:
    model.eval()

    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0).numpy()

In [ ]:
print(task.target_col)

num_of_adverse_events


# Training

In [ ]:
import numpy as np
import torch
from typing import Dict, List, Tuple
from torch_geometric.data import HeteroData

def mask_attributes(batch: HeteroData,
                    maskable_attributes: Dict[str, Dict[str, List[str]]],
                    p_mask=0.3,
                    device="cuda") -> Tuple[HeteroData, Dict]:
    """
    Maschera dinamicamente alcune feature nel batch, e restituisce anche i valori originali mascherati.
    """
    target_values = {}  # per il decoder: quali campi/righe sono stati mascherati e con quali valori originali

    for node_type, type_dict in maskable_attributes.items():
        if node_type not in batch.node_types:
            print(f"node type {node_type} non trovato, questo grafo ha nodi {batch.node_types}")
            continue

        for attr_type, cols in type_dict.items():
            for col in cols:

                table_columns = []
                for stype, cols_in in batch[node_type].tf.col_names_dict.items():
                    table_columns.extend(cols_in)

                if col not in table_columns:
                    print(f"non abbiamo trovato la colonna {col} nella tabella {node_type}")
                    continue  # Colonna non disponibile nel batch

                # --- batch-only: lavora sul DF del batch, non sul DB globale ---
                table = batch[node_type].tf.table
                df = table.df.copy(deep=True)

                # ID globali presenti nel batch e maschera Bernoulli batch-local
                local_ids = batch[node_type].n_id.cpu().numpy()
                if local_ids.size == 0:
                    continue
                bern = (torch.rand(len(local_ids), device=device) < p_mask).cpu().numpy()
                masked_pos = np.nonzero(bern)[0]  # indici batch-local
                if masked_pos.size == 0:
                    continue
                masked_global = local_ids[masked_pos]

                # salva indici batch-local e valori originali
                target_values[(node_type, col)] = {
                    "indices": masked_pos.tolist(),
                    "values": df.loc[masked_global, col].tolist()
                }

                # applica sentinel SOLO nel batch
                if attr_type == "categorical":
                    df.loc[masked_global, col] = 0
                elif attr_type == "numerical":
                    df.loc[masked_global, col] = 0.0
                else:
                    raise ValueError(f"Tipo non supportato: {attr_type}")

                # scrivi indietro SOLO al batch
                table.df = df

    return batch, target_values


In [ ]:
class MAPDecoder(nn.Module):
    def __init__(self, encoder_out_dim: int, hidden_dim: int = 128):
        super().__init__()
        self.decoder_dict = nn.ModuleDict()
        self.encoder_out_dim = encoder_out_dim
        self.hidden_dim = hidden_dim

    def add_decoder(self, name: str, out_dim: int, task: str):
        if task == "regression":
            self.decoder_dict[name] = nn.Linear(self.encoder_out_dim, out_dim)
        elif task == "classification":
            self.decoder_dict[name] = nn.Linear(self.encoder_out_dim, out_dim)

    def forward(self, z_dict, batch, mask_info):
        losses = []

        for (node_type, col), info in mask_info.items():
            z = z_dict[node_type]

            # Converto in tensori PyTorch
            indices = torch.tensor(info["indices"], device=z.device)
            true_vals = torch.tensor(info["values"], device=z.device)

            # Estraggo i nodi mascherati
            z_masked = z[indices]
            decoder = self.decoder_dict[f"{node_type}__{col}"]
            pred = decoder(z_masked)

            if pred.numel() == 0:
                continue  # salta se batch vuoto

            #lss diversa per regressione o classificazione
            if true_vals.dtype in [torch.float, torch.float32, torch.float64]:
                loss = F.mse_loss(pred.squeeze(), true_vals.float())
            else:
                loss = F.cross_entropy(pred, true_vals.long())

            losses.append(loss)

        if len(losses) == 0:
            return torch.tensor(0.0, requires_grad=True, device=z.device)

        return sum(losses)


In [ ]:
from collections import defaultdict

def extract_categorical_values_from_db(db, maskable_attributes):
    value_dict = defaultdict(lambda: defaultdict(set))

    for node_type, type_dict in maskable_attributes.items():
        if node_type not in db.table_dict:
            print(f"Tabella {node_type} non trovata nel db.")
            continue

        table = db.table_dict[node_type]
        df = table.df

        for col in type_dict.get("categorical", []):
            if col not in df.columns:
                print(f"Colonna {col} non trovata nella tabella {node_type}")
                continue

            unique_vals = df[col].dropna().unique()
            value_dict[node_type][col].update(unique_vals)

    # Converte in liste ordinate per sicurezza
    value_dict_final = {
        node: {col: sorted(list(vals)) for col, vals in col_dict.items()}
        for node, col_dict in value_dict.items()
    }

    return value_dict_final


In [ ]:
# cat_values = extract_categorical_values_from_db(db, maskable_attributes)
# print(cat_values)

In [ ]:

def train_map(model, loader_dict, maskable_attributes, encoder_out_dim: int, device: str, cat_values ,epochs: int = 20):
    model.train()
    decoder = MAPDecoder(encoder_out_dim)

    #Inizializzazione decoder per ogni colonna
    for node_type, type_dict in maskable_attributes.items():
      for col in type_dict.get("categorical", []):
          try:
              out_dim = len(cat_values[node_type][col])#qua prendo il numero di possibili valori
              decoder.add_decoder(f"{node_type}__{col}", out_dim=out_dim, task="classification")
              #print(f"Aggiunto decoder per {node_type}__{col} con out_dim={out_dim}")
          except KeyError:
              print(f"Nessun valore trovato per {node_type}__{col}, decoder non aggiunto")

      for col in type_dict.get("numerical", []):
          decoder.add_decoder(f"{node_type}__{col}", out_dim=1, task="regression")


    decoder.to(device)
    optimizer = torch.optim.Adam(list(model.parameters()) + list(decoder.parameters()), lr=1e-3)

    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        for batch in loader_dict["train"]:
            batch = batch.to(device)
            batch, mask_info = mask_attributes(batch, maskable_attributes)
            z_dict = model.encode_node_types(batch, node_types=list(maskable_attributes.keys()))

            loss = decoder(z_dict, batch, mask_info)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"[MAP] Epoch {epoch:02d} | Loss: {total_loss:.4f}")

    return model

In [ ]:
# for batch in loader_dict["train"]:
#     batch = batch.to(device)
#     batch, mask_info = mask_attributes(batch, maskable_attributes)
#     print(mask_info)
#     for (node_type, col), info in mask_info.items():
#       #z = z_dict[node_type]

#             # Converto in tensori PyTorch
#       indices = torch.tensor(info["indices"], device="cuda")
#       #true_vals = torch.tensor(info["values"], device="cuda")
#       raw_vals = info["values"]

#       if any(isinstance(v, str) for v in raw_vals):
#           try:
#               classes = mask_info[node_type][col]
#               str_to_idx = {v: i for i, v in enumerate(classes)}
#               idx_vals = [str_to_idx[val] if isinstance(val, str) else 0 for val in raw_vals]  # fallback 0
#               true_vals = torch.tensor(idx_vals, device=z.device)
#           except KeyError:

#               continue

#       else:
#           try:
#               float_vals = [float(v) if isinstance(v, (int, float, str)) and str(v).strip() != "" else 0.0 for v in raw_vals]
#               true_vals = torch.tensor(float_vals, device=z.device)
#           except Exception as e:
#               print(f"[⚠️] Errore nella conversione numerica per {node_type}.{col}: {e}")
#               continue

#       print(f"indices {indices}")
#       print(f"true values{true_vals} ")
#     break

In [ ]:
# print(f"{cat_values}")

In [ ]:
loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256)

In [ ]:
# train_map(model, loader_dict, maskable_attributes, encoder_out_dim=128, device=device, cat_values = cat_values, epochs=20)

In [ ]:
model = Model(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="max",
    norm="batch_norm",
).to(device)

# maskable_attributes = {
#     "studies": {
#         "categorical": ["study_type", "phase", "is_fda_regulated_drug", "is_fda_regulated_device"],
#         "numerical": ["enrollment"]
#     },
#     "outcomes": {
#         "categorical": ["outcome_type"]
#     },
#     "reported_event_totals": {
#         "categorical": ["event_type", "classification"]
#     }
# }



In [ ]:
def evaluate_performance(pred: np.ndarray, target_table, metrics) -> dict:
    """Custom evaluation function to replace task.evaluate."""
    target = target_table.df[task.target_col].to_numpy()

    if len(pred) != len(target):
        raise ValueError(
            f"The length of pred and target must be the same (got "
            f"{len(pred)} and {len(target)}, respectively)."
        )

    results = {}
    for metric_fn in metrics:
        if metric_fn.__name__ == "rmse":
            results["rmse"] = np.sqrt(np.mean((target - pred)**2))
        else:
            results[metric_fn.__name__] = metric_fn(target, pred)

    return results

In [ ]:
import numpy as np

def evaluate_on_train_during_training() -> float:
    model.eval()
    pred_list, target_list = [], []

    for batch in loader_dict["train"]:
        batch = batch.to(device)
        pred = model(batch, task.entity_table)
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
        target_list.append(batch[task.entity_table].y.detach().cpu())

    pred_all = torch.cat(pred_list, dim=0).numpy()
    target_all = torch.cat(target_list, dim=0).numpy()

    mae = np.mean(np.abs(pred_all - target_all))
    return mae


In [ ]:
model = Model(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="max",
    norm="batch_norm",
).to(device)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.0005,
    weight_decay=0
)

scheduler = CosineAnnealingLR(optimizer, T_max=100)

loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256)


epochs = 100

state_dict = None
test_table = task.get_table("test", mask_input_cols=False)
best_val_metric = -math.inf if higher_is_better else math.inf
best_test_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train(model, optimizer, loader_dict=loader_dict)

    train_pred = test(model, loader_dict["train"])
    train_metrics = evaluate_performance(train_pred, train_table, task.metrics)
    train_mae_preciso = evaluate_on_full_train(model, loader_dict["train"])

    val_pred = test(model, loader_dict["val"])
    val_metrics = evaluate_performance(val_pred, val_table, task.metrics)

    test_pred = test(model, loader_dict["test"])
    test_metrics = evaluate_performance(test_pred, test_table, task.metrics)

    scheduler.step(val_metrics[tune_metric])

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
            not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())

    #test:
    if (higher_is_better and test_metrics[tune_metric] > best_test_metric) or (
            not higher_is_better and test_metrics[tune_metric] < best_test_metric
    ):
        best_test_metric = test_metrics[tune_metric]
        state_dict_test = copy.deepcopy(model.state_dict())

    current_lr = optimizer.param_groups[0]["lr"]
    print(f"Epoch: {epoch:02d}, Train {tune_metric}: {train_mae_preciso:.2f}, Validation {tune_metric}: {val_metrics[tune_metric]:.2f}, Test {tune_metric}: {test_metrics[tune_metric]:.2f}, LR: {current_lr:.6f}")

    #early_stopping(val_metrics[tune_metric], model)

    # if early_stopping.early_stop:
    #     print(f"Early stopping triggered at epoch {epoch}")
    #     break
print(f"best validation results: {best_val_metric}")
print(f"best test results: {best_test_metric}")


100%|██████████| 85/85 [00:18<00:00,  4.64it/s]


NameError: name 'evaluate_on_full_train' is not defined